In [ ]:
# Exploratory Data Analysis (EDA) using the Sentiment140 dataset

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
import pandas as pd

# Load the datasets
adidas_nike_df = pd.read_csv(r'C:\Users\User\Documents\Brain Station\Data Science\Capstone\capstone_social_listening_GC\data\Adidas Vs Nike.csv')
nike_sales_df = pd.read_csv(r'C:\Users\User\Documents\Brain Station\Data Science\Capstone\capstone_social_listening_GC\data\nike_shoes_sales.csv')
sentiment_df = pd.read_csv(r'C:\Users\User\Documents\Brain Station\Data Science\Capstone\capstone_social_listening_GC\data\training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=['target', 'ids', 'date', 'flag', 'user', 'text'])

# Display the first few rows of each dataset
print("Adidas vs Nike Data:")
print(adidas_nike_df.head(), "\n")

print("Nike Shoes Sales Data:")
print(nike_sales_df.head(), "\n")

print("Sentiment140 Data:")
print(sentiment_df.head())


Adidas vs Nike Data:
                                        Product Name Product ID  \
0  Women's adidas Originals NMD_Racer Primeknit S...     AH2430   
1               Women's adidas Originals Sleek Shoes     G27341   
2                  Women's adidas Swim Puka Slippers     CM0081   
3   Women's adidas Sport Inspired Questar Ride Shoes     B44832   
4           Women's adidas Originals Taekwondo Shoes     D98205   

   Listing Price  Sale Price  Discount                    Brand  \
0          14999        7499        50  Adidas Adidas ORIGINALS   
1           7599        3799        50         Adidas ORIGINALS   
2            999         599        40        Adidas CORE / NEO   
3           6999        3499        50        Adidas CORE / NEO   
4           7999        3999        50         Adidas ORIGINALS   

                                         Description  Rating  Reviews  \
0  Channeling the streamlined look of an '80s rac...     4.8       41   
1  A modern take on adidas 

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import nltk

# Preprocessing
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

sentiment_df['text'] = sentiment_df['text'].apply(preprocess_text)

# Convert target labels to binary (0 = negative, 1 = positive)
sentiment_df['target'] = sentiment_df['target'].apply(lambda x: 1 if x == 4 else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sentiment_df['text'], sentiment_df['target'], test_size=0.2, random_state=42)

# Convert text data into TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.79      0.75      0.77    159494
           1       0.76      0.80      0.78    160506

    accuracy                           0.78    320000
   macro avg       0.78      0.78      0.78    320000
weighted avg       0.78      0.78      0.78    320000



In [26]:
print(adidas_nike_df.columns)


Index(['Product Name', 'Product ID', 'Listing Price', 'Sale Price', 'Discount',
       'Brand', 'Description', 'Rating', 'Reviews', 'Last Visited'],
      dtype='object')


In [18]:
adidas_nike_df['preprocessed_text'] = adidas_nike_df['text'].apply(preprocess_text)
adidas_nike_tfidf = vectorizer.transform(adidas_nike_df['preprocessed_text'])
adidas_nike_df['sentiment'] = model.predict(adidas_nike_tfidf)

print(adidas_nike_df[['text', 'sentiment']].head())


KeyError: 'text'

In [20]:
from sklearn.decomposition import LatentDirichletAllocation

# Perform topic modeling
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda_topics = lda.fit_transform(adidas_nike_tfidf)

# Display the top words in each topic
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

tf_feature_names = vectorizer.get_feature_names_out()
display_topics(lda, tf_feature_names, 10)


NameError: name 'adidas_nike_tfidf' is not defined

In [22]:
# Merge the sentiment data with sales data (assuming there's a common time frame or other linking key)
merged_df = pd.merge(adidas_nike_df, nike_sales_df, on='date_column_here')

# Analyze correlations
print(merged_df.corr())

# Visualize correlations
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(merged_df.corr(), annot=True)
plt.show()


KeyError: 'date_column_here'